In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spotipy
import time
import random

%matplotlib inline

from pandas import json_normalize
from random import randint, choice
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
string = open("access.txt","r").read()
access={}

for line in string.split('\n'):
    if len(line) > 0:
        access[line.split(':')[0]]=line.split(':')[1].strip()

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=access['clientid'],
                                                           client_secret=access['clientsecret']))

In [3]:
df=pd.read_csv('tracks_w_audio.csv')
hot100=pd.read_csv('BillboardHot100.csv')
popv=pd.read_csv('Popvortex100.csv')

In [4]:
df_audio=df.select_dtypes(include=np.number).drop(columns='cluster')
df_audio['popularity']=df_audio['popularity']**2

In [28]:
scaler = StandardScaler()
audio_scaled = scaler.fit_transform(df_audio)

In [33]:
kmeans = KMeans(n_clusters=27, random_state=42)
kmeans.fit(audio_scaled)

KMeans(n_clusters=27, random_state=42)

In [34]:
clusters = kmeans.predict(audio_scaled)

In [35]:
def get_info(song):
        song = sp.search(q=song, limit=1)
        song_info = pd.DataFrame(sp.audio_features(song["tracks"]["items"][0]["uri"]))
        print(song_info)


In [66]:
def recommendation1(df1, df2):
    name = input("Enter a song that you like and we recommend a similar song 🎶 \n")
    user_input = name.lower()
    
    df1['song'] = df1['song'].str.lower()
    df2['song'] = df2['song'].str.lower()
    
    if user_input in df1['song'].values:
        df = df1[df1['song'] != user_input]
    elif user_input in df2['song'].values:
        df = df2[df2['song'] != user_input]
    else:
        return user_input, False
    
    if user_input in df1['song'].values and user_input in df2['song'].values:
        df = pd.concat([df1, df2], ignore_index=True)
    
    rec = choice(df.index)
    song = df.loc[rec, 'song']
    artist = df.loc[rec, 'artist']
    print(f"So you like hot songs huh?\n Well, let's find you another one!")
    time.sleep(2)
    print(f"Here is another song that is quite hot right now:\n{song} by {artist}")
    return user_input, True

In [68]:
recommendation1(hot100,popv)

Enter a song that you like and we recommend a similar song 🎶 
mozart


('mozart', False)

In [73]:
song, popular = recommendation1(hot100,popv)
    
if not popular:
    time.sleep(0.5)
    print("Let's find you a similar song!")
    song = sp.search(q=song, limit=1)
    song_info = json_normalize(song['tracks']['items'])[['popularity','uri']]
    song_info['popularity']=song_info['popularity']**2
    song_info= pd.concat([song_info[['popularity']],json_normalize(sp.audio_features(song_info['uri'])).select_dtypes(include=np.number)],axis=1)
        
    song_scaled= scaler.transform(song_info)
    song_cluster = kmeans.predict(song_scaled)
    rec_song_info = df[df['cluster']==int(song_cluster)].sample(n=1)
    rec_song = rec_song_info.iloc[0]['song_name']
    rec_artist = rec_song_info.iloc[0]['artists']
    time.sleep(randint(1000,5000)/1000)
    print(f'Okay, I think I got it.')
    time.sleep(1)
    print(f'What do you think of this song?')
    time.sleep(0.5)
    print(f"{rec_song} by {rec_artist}")

print('Did you like the song?')
satisfaction = input("Y/N").lower()
unsatisfaction = {}
if satisfaction=='y':
    print('So happy to hear that. Wanna try again?')
    rerun = input("Y/N").lower()
if satisfaction=='n':
    print('So sorry to hear that. Could you please give us another chance?')
    
    
    


Enter a song that you like and we recommend a similar song 🎶 
mozart
Let's find you a similar song!
Okay, I think I got it.
What do you think of this song?
Tannhäuser, WWV 70, Act III: Allmächt'ge Jungfrau, hör mein Flehen! by Richard Wagner, Lise Davidsen, Philharmonia Orchestra, Esa-Pekka Salonen


In [ ]:
#song = sp.search(q='all too well', limit=1)
#song["tracks"]["items"]

In [ ]:
#song_info = json_normalize(song['tracks']['items'])[['popularity','uri']]
#song_info['popularity']=song_info['popularity']**2

In [ ]:
#song_info= pd.concat([song_info[['popularity']],json_normalize(sp.audio_features(song_info['uri'])).select_dtypes(include=np.number)],axis=1)

In [ ]:
#song_info

In [ ]:
#song_scaled= scaler.transform(song_info)
#pd.DataFrame(song_scaled, columns=song_info.columns)